# 3주차 Day3 — RAG 시스템 데모

> **블록1 체험용** | 강사 전용 데모 노트북
>
> 수강생에게는 `ask("질문")` 셀과 출력 결과만 보여줍니다.
> 
> **강의 전 준비**: 아래 `[준비]` 셀 2개를 미리 실행해두세요.

---
## [준비] 시스템 초기화
> 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.

In [ ]:
# ============================================================
# [준비 1/2] 패키지 설치 + 환경설정
# 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.
# ============================================================

# pip install: 필요한 외부 라이브러리를 인터넷에서 내려받아 설치하는 명령어
# langchain-openai: OpenAI GPT + 임베딩을 langchain에서 쓸 수 있게 해주는 패키지
# langchain-community: PDF 로더, ChromaDB 연결 등 커뮤니티 확장 도구 모음
# langchain-text-splitters: 긴 문서를 작은 조각으로 잘라주는 도구 (별도 패키지)
# chromadb: 텍스트를 벡터로 저장하고 검색하는 데이터베이스
# pypdf: PDF 파일을 읽어서 텍스트로 변환해주는 라이브러리
# tiktoken: OpenAI 모델의 토큰 수를 계산하는 도구
!pip install -q langchain langchain-openai langchain-community langchain-text-splitters chromadb pypdf tiktoken

import os
from IPython.display import display, HTML  # 노트북에서 HTML을 예쁘게 출력하기 위한 도구

# === OpenAI API 키 설정 ===
# API 키: OpenAI GPT 서비스를 사용하기 위한 인증 비밀번호
# 아래 주석(#)을 제거하고 발급받은 키를 입력하세요
# os.environ["OPENAI_API_KEY"] = "sk-..."

# .env 파일에서 키를 불러오는 방법 (파일에 OPENAI_API_KEY=sk-... 형태로 저장)
try:
    from dotenv import load_dotenv  # .env 파일을 읽어 환경변수로 등록해주는 도구
    load_dotenv()
except ImportError:
    pass  # dotenv가 없으면 그냥 넘어감

# assert: 조건이 False이면 즉시 에러를 발생시키는 검증 구문
assert os.environ.get("OPENAI_API_KEY"), "OPENAI_API_KEY가 설정되지 않았습니다!"
print("환경설정 완료")

In [4]:
# ============================================================
# [준비 2/2] 벡터DB 구축 + RAG 체인 생성
# 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.
# ============================================================

import os
from pathlib import Path  # 파일 경로를 다루는 파이썬 기본 모듈

# 각 도구를 해당 패키지에서 불러옴 (import)
from langchain_community.document_loaders import PyPDFLoader          # PDF 파일을 읽는 도구
from langchain_text_splitters import RecursiveCharacterTextSplitter   # 문서를 조각으로 자르는 도구
from langchain_community.vectorstores import Chroma                   # 벡터를 저장·검색하는 DB
from langchain_openai import OpenAIEmbeddings, ChatOpenAI             # OpenAI 임베딩 + GPT 모델
from langchain_core.prompts import PromptTemplate                     # 프롬프트 템플릿 도구
from IPython.display import display, HTML  # 노트북에서 HTML 결과를 예쁘게 출력하는 도구


# --- 0. 작업 디렉토리 자동 설정 ---
# 노트북을 어디서 실행하든 PDF 파일이 있는 폴더를 자동으로 찾아서 이동
# os.walk: 지정한 경로 아래 모든 폴더와 파일을 순서대로 탐색하는 함수
pdf_name = "제품사양서_스마트냉장고_RF9000.pdf"
found = False
for root, dirs, files in os.walk(str(Path.cwd())):
    if pdf_name in files:       # PDF가 있는 폴더를 발견하면
        os.chdir(root)          # 작업 디렉토리를 그 폴더로 변경
        print(f"작업 디렉토리: {root}")
        found = True
        break                   # 찾았으니 탐색 중단
if not found:
    print(f"PDF를 찾지 못했습니다. 현재 위치: {os.getcwd()}")


# --- 1. PDF 로딩 ---
# PyPDFLoader: PDF를 페이지 단위로 읽어서 텍스트를 추출하는 도구
pdf_files = [
    "제품사양서_스마트냉장고_RF9000.pdf",
    "시험성적서_스마트냉장고_RF9000.pdf"
]

all_docs = []                       # 모든 페이지를 담을 빈 리스트
for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)  # PDF 읽기 도구 생성
    docs = loader.load()            # 실제로 PDF를 읽어 페이지별로 분리
    all_docs.extend(docs)           # 전체 목록에 추가
    print(f"{pdf_path} → {len(docs)}페이지 로딩")

print(f"총 {len(all_docs)}페이지 로딩 완료")


# --- 2. 청킹 (문서 조각 분할) ---
# chunk_size=500: 한 조각의 최대 글자 수
# chunk_overlap=50: 조각 경계에서 문맥이 끊기지 않도록 50자씩 겹치게 설정
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)  # 전체 문서를 조각으로 분할
print(f"{len(chunks)}개 청크로 분할")


# --- 3. 벡터DB 구축 ---
# OpenAIEmbeddings: 텍스트를 숫자 배열(벡터)로 변환하는 OpenAI 임베딩 모델
# 임베딩된 벡터들을 ChromaDB에 저장해서 유사도 검색이 가능해짐
print("임베딩 중... (처음 실행 시 수십 초 소요)")
embeddings = OpenAIEmbeddings()

# Chroma.from_documents: 문서 조각들을 벡터로 변환해서 ChromaDB에 한꺼번에 저장
vectorstore = Chroma.from_documents(chunks, embeddings)

# as_retriever: 벡터DB를 검색기로 변환. k=3이면 가장 유사한 조각 3개를 검색
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("벡터DB 구축 완료")


# --- 4. LLM + 커스텀 프롬프트 설정 ---
# ChatOpenAI: 질문을 받아 답변을 생성하는 GPT 언어 모델
# gpt-4o-mini: 빠르고 저렴한 GPT 모델
# temperature=0: 창의성을 끄고 가장 확실한 답변만 출력
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# PromptTemplate: 변수({context}, {query})가 포함된 프롬프트 틀
custom_prompt = PromptTemplate(
    template="""당신은 제조업 기술문서 전문 AI 어시스턴트입니다.
다음 문서 내용만을 참고하여 질문에 정확하게 답하세요.

문서 내용:
{context}

질문: {query}

답변 규칙:
1. 반드시 문서에 있는 정보만 사용하세요
2. 문서에 없는 내용이면 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요
4. 가능하면 출처 페이지를 언급하세요

답변:""",
    input_variables=["context", "query"]  # 자동으로 채워질 변수 2개
)


# --- 5. 출력 함수 정의 ---
def ask(query):
    """질문을 입력하면 답변과 출처를 함께 출력하는 함수"""

    # retriever.invoke: 질문과 의미가 비슷한 문서 조각 3개를 벡터DB에서 검색
    docs = retriever.invoke(query)

    # 검색된 조각들을 하나의 텍스트로 합침 (LLM에 넘겨줄 참고 문서)
    context = "\n\n".join(doc.page_content for doc in docs)

    # PromptTemplate.format(): {context}와 {query}에 실제 값을 채워 완성된 프롬프트 생성
    prompt_text = custom_prompt.format(context=context, query=query)

    # llm.invoke: 프롬프트를 LLM에 전달하고 답변을 받음
    # .content: 응답 객체에서 텍스트 내용만 꺼냄
    answer = llm.invoke(prompt_text).content

    # 출처 정리 (중복 제거)
    sources = []
    seen = set()  # 이미 추가한 출처를 기억해서 중복 방지
    for doc in docs:
        src = doc.metadata.get("source", "알 수 없음").split("/")[-1]  # 파일명만 추출
        page = doc.metadata.get("page", -1) + 1  # 페이지는 0부터 시작이므로 +1
        key = f"{src}_p{page}"
        if key not in seen:
            seen.add(key)
            snippet = doc.page_content[:80].replace("\n", " ").strip()
            sources.append((src, page, snippet))

    # 출처 목록을 HTML 카드 형태로 만듦
    src_html = ""
    for i, (src, page, snippet) in enumerate(sources, 1):
        src_html += f"""
        <div style='margin:4px 0; padding:6px 12px; background:#f0e6ff; border-radius:6px; font-size:13px;'>
        <b>[{i}]</b> {src} — <b>p.{page}</b>
        <br><span style='color:#8888aa; font-size:11px;'>"...{snippet}..."</span>
        </div>"""

    # 질문·답변·출처를 하나의 HTML 카드로 조합해서 출력
    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='background:white; border:2px solid #E8DEF8; padding:16px 20px;'>
    <div style='font-size:15px; line-height:1.6; color:#1A1A2E;'>
    <b>답변:</b><br>{answer}
    </div>
    </div>
    <div style='background:#f5f1fa; padding:12px 20px; border-radius:0 0 10px 10px; border:1px solid #E8DEF8; border-top:none;'>
    <div style='font-size:13px; color:#7B2FBE; font-weight:bold; margin-bottom:6px;'>참조 출처:</div>
    {src_html}
    </div>
    </div>
    """
    display(HTML(html))


def compare(query):
    """같은 질문을 규칙 없는 기본 체인과 규칙 있는 커스텀 체인에 동시에 던져 결과를 비교"""

    docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in docs)

    # 기본 프롬프트: 규칙 없이 그냥 문서 참고해서 답하라고만 지시
    basic_prompt_text = f"""다음 문서를 참고하여 질문에 답하세요.

문서 내용:
{context}

질문: {query}

답변:"""

    # 커스텀 프롬프트: 규칙 4가지를 명시해서 더 정확한 답변을 유도
    custom_prompt_text = custom_prompt.format(context=context, query=query)

    r_basic = llm.invoke(basic_prompt_text).content   # 기본 체인 답변
    r_custom = llm.invoke(custom_prompt_text).content  # 커스텀 체인 답변

    # 두 답변을 나란히 놓고 비교하는 HTML 레이아웃
    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='display:flex; gap:0;'>
    <div style='flex:1; background:#fff; border:2px solid #ccc; padding:14px; border-radius:0 0 0 10px;'>
    <div style='background:#8888aa; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>기본 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#4A4A6A;'>{r_basic}</div>
    </div>
    <div style='flex:1; background:#fff; border:2px solid #7B2FBE; padding:14px; border-radius:0 0 10px 0;'>
    <div style='background:#7B2FBE; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>커스텀 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#1A1A2E; font-weight:500;'>{r_custom}</div>
    </div>
    </div>
    </div>
    """
    display(HTML(html))


print("시스템 준비 완료")
print('ask("질문") → 답변 + 출처')
print('compare("질문") → 기본 vs 커스텀 비교')

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'


작업 디렉토리: c:\Users\User\Downloads\엘리스\01_submit_github\2.강의자료_및_실습_콘텐츠_제작\실습_자료(jupyter_notebook,pdf)
제품사양서_스마트냉장고_RF9000.pdf → 4페이지 로딩
시험성적서_스마트냉장고_RF9000.pdf → 3페이지 로딩
총 7페이지 로딩 완료
11개 청크로 분할
임베딩 중... (처음 실행 시 수십 초 소요)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

---
---
# 데모 시작

> **여기부터 수강생에게 보여줍니다.**
>
> `ask("질문")` 한 줄이 전부입니다. 코드는 신경쓰지 마세요, 결과만 보세요.

## 데모 ① 사양서에 질문하기

In [ ]:
ask("RF9000의 총 용량은 얼마인가요?")

In [ ]:
ask("정격 소비전력은?")

In [ ]:
ask("E3 에러코드는 무엇인가요? 원인과 조치 방법을 알려주세요.")

## 데모 ② 시험성적서에도 질문해보자

In [ ]:
ask("에너지효율 등급은 몇 등급인가요?")

In [ ]:
ask("냉장 운전 소음 측정값은?")

## 데모 ③ 문서에 없는 질문을 하면?

In [ ]:
ask("RF9000의 판매 가격은 얼마인가요?")

In [ ]:
ask("RF9000과 RF8000의 성능 차이는?")

## 데모 ④ 수강생 자유 질문

> **수강생에게 질문을 받아서 아래 셀에 타이핑하세요.**

In [ ]:
ask("")

In [ ]:
ask("")

In [ ]:
ask("")

## 데모 ⑤ 같은 질문, 다른 답변?

> 프롬프트에 규칙 하나를 추가했을 뿐인데 이만큼 달라집니다.
> → 블록 2에서 자세히 다룹니다.

In [ ]:
compare("냉장 용량과 냉동 용량은?")

In [ ]:
compare("절연저항 시험 결과는?")

---
## 강사 전용: 추가 질의 모음

> 시간이 남거나 수강생 질문에 대응할 때 사용

In [ ]:
# 교차 참조 질문
ask("IEC 60335 기준으로 절연저항은 적합한가요?")

In [ ]:
# 계산이 필요한 질문
ask("냉장실과 냉동실의 용량 차이는 얼마인가요?")

In [ ]:
# 목록 질문
ask("E1부터 E9까지 모든 에러코드의 명칭을 알려주세요.")

In [ ]:
# 기능 관련
ask("FoodAI 식품인식 시스템의 인식 정확도는?")

In [ ]:
# 유지보수 관련
ask("탈취 필터 교체 주기는 얼마인가요?")

In [ ]:
# EMC 관련
ask("전도성 방출 시험 결과가 기준 이내인가요?")